In [6]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from  langchain_classic.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI
from langchain_google_genai import GoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain_classic.chains import GraphQAChain

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


In [7]:
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = OpenAI(model='gpt-3.5-turbo-instruct',api_key=OPENAI_API_KEY)

In [8]:
llm

OpenAI(client=<openai.resources.completions.Completions object at 0x11ce6e900>, async_client=<openai.resources.completions.AsyncCompletions object at 0x11ce6eba0>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [10]:
def graphrag():
        with open('hybrid_rag.txt', 'r') as file:
            content = file.read()

        documents = [Document(page_content=content)]
        llm_transformer = LLMGraphTransformer(llm=llm)
        graph_documents = llm_transformer.convert_to_graph_documents(documents)

        graph = NetworkxEntityGraph()

        # Add nodes to the graph
        for node in graph_documents[0].nodes:
            graph.add_node(node.id)

        # Add edges to the graph
        for edge in graph_documents[0].relationships:
            graph._graph.add_edge(
                    edge.source.id,
                    edge.target.id,
                    relation=edge.type,
                )

            graph._graph.add_edge(
                    edge.target.id,
                    edge.source.id,
                    relation=edge.type+" by",
                )

        chain = GraphQAChain.from_llm(
            llm=llm, 
            graph=graph, 
            verbose=True
        )
        
        return chain

In [16]:
def rag():
        #Document Loader
        loader = TextLoader('hybrid_rag.txt')
        data = loader.load()

        #Document Transformer
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        texts = text_splitter.split_documents(data)

        #Vector DB
        docsearch = Chroma.from_documents(texts, embeddings)

        #Hyperparameters to know
        retriever = docsearch.as_retriever(search_type='similarity_score_threshold',search_kwargs={"k": 7,"score_threshold":0.1})
        qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
        
        return qa

In [17]:
standard_rag = rag()
graph_rag = graphrag()

In [18]:
def hybrid_rag(query,standard_rag,graph_rag):
    result1 = standard_rag.run(query)
    
    print("Standard RAG:",result1)
    result2 = graph_rag.run(query)
    
    
    print("Graph RAG:",result2)
    prompt = "Generate a final answer using the two context given : Context 1: {} \n Context 2: {} \n Question: {}".format(result1,result2,query)
    #return llm(prompt)
    return llm.generate([prompt]).generations[0][0].text

query = "Some characteristics of hybrid rag"
hybrid = hybrid_rag(query,standard_rag,graph_rag)
print("Hybrid:",hybrid)

/Users/piyushpande/Desktop/Agentic_ai/Graph_RAG/graphRAG/.venv/lib/python3.13/site-packages/langchain_core/vectorstores/base.py:1043: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'source': 'hybrid_rag.txt'}, page_content='The amalgamation of these two contexts allows us to leverage the strengths of both approaches. The VectorRAG component provides a broad, similarity-based retrieval of relevant information, while the GraphRAG element contributes structured, relationship-rich contextual data. This combined context is then utilized as input for a LLM to generate the final responses. Details on the implementation of the HybridRAG will be provided in Section 4.4.'), 0.23384270235768823), (Document(metadata={'source': 'hybrid_rag.txt'}, page_content='The amalgamation of these two contexts allows us to leverage the strengths of both approaches. The VectorRAG component provides a broad, similarity-based retrieval of relevant information, while the GraphRAG e

Standard RAG:  The hybrid RAG technique integrates traditional vector-based retrieval with a KG-based retrieval system, combining broad similarity-based retrieval with structured, relationship-rich contextual data. It is utilized in a LLM to generate final responses and its implementation is detailed in section 4.4.


> Entering new GraphQAChain chain...
Entities Extracted:
 NONE
Full Context:


> Finished chain.
Graph RAG:  Hybrid rag is a crossbreed between a ragdoll and another breed of cat. It can have a variety of coat colors and patterns, and typically has the laid-back and affectionate personality of a ragdoll.
Hybrid: 

The hybrid RAG technique is a method of integrating traditional vector-based retrieval with a KG-based retrieval system. It combines broad similarity-based retrieval with structured, relationship-rich contextual data, and is used in a LLM to generate final responses. This technique is detailed in section 4.4. Additionally, a hybrid rag is a crossbreed between a 

In [19]:
query = "Some characteristics of GraphRAG"
hybrid = hybrid_rag(query,standard_rag,graph_rag)
print("Hybrid:",hybrid)

/Users/piyushpande/Desktop/Agentic_ai/Graph_RAG/graphRAG/.venv/lib/python3.13/site-packages/langchain_core/vectorstores/base.py:1043: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'source': 'hybrid_rag.txt'}, page_content='A schematic diagram of the retrieval methodology of GraphRAG is given in Figure 2. Here we first write a prompt to clean the data and then write another prompt in the second stage to create knowledge triplets along with metadata. It will be covered in more detail in section 4.1\n\nFor the HybridRAG technique, we propose to integrate the aforementioned two distinct RAG techniques: VectorRAG and GraphRAG. This integration involves a systematic combination of contextual information retrieved from both the traditional vector-based retrieval mechanism and the KG-based retrieval system, the latter of which was constructed specifically for this study.'), 0.4383668596651692), (Document(metadata={'source': 'hybrid_rag.txt'}, page_content='A s

Standard RAG:  The retrieval methodology of GraphRAG involves a systematic combination of contextual information retrieved from both the traditional vector-based retrieval mechanism and the KG-based retrieval system. It utilizes a subgraph of relevant nodes and edges from the KG and combines it with the language model's internal knowledge. GraphRAG also leverages metadata information to selectively filter and retrieve only relevant document segments.


> Entering new GraphQAChain chain...
Entities Extracted:
 GraphRAG
Full Context:
GraphRAG IS_DIFFERENT_TO VectorRAG
GraphRAG HAS_INTEGRATION by language model

> Finished chain.
Graph RAG: 

I don't know.
Hybrid: 

GraphRAG is a retrieval methodology that combines contextual information from both traditional vector-based retrieval and a KG-based retrieval system. It utilizes a subgraph of relevant nodes and edges from the KG and integrates it with the internal knowledge of a language model. Additionally, GraphRAG leverages metadata infor